In [1]:
!pip install transformers
!pip install tensorflow-gpu
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, set_seed
from tensorflow.keras.optimizers import Adam

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [44 lines of output]
  Traceback (most recent call last):
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\requirements.py", line 35, in __init__
      parsed = _parse_requirement(requirement_string)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 64, in parse_requirement
      return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_vendor\packaging\_parser.py", line 82, in _parse_requirement
      url, specifier, marker = _parse_requirement_details(tokenizer)
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\marks\anaconda3\Lib\site-packages\setuptools\_v

C:\Users\marks\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\marks\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
set_seed(123)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token 

In [3]:
model = TFGPT2LMHeadModel.from_pretrained("distilgpt2")

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [4]:
input_sequences = []
target_sequences = []

with open("dataset.txt", "r", encoding="utf-8") as file:
    for line in file:
        question, answer = line.strip().split('\t')
        combined_text = question + tokenizer.eos_token + answer
        encoded = tokenizer.encode_plus(combined_text, max_length=64, truncation=True, padding="max_length", return_tensors="tf")
        input_sequences.append(encoded['input_ids'][0])
        target_sequences.append(tf.concat([encoded['input_ids'][0][1:], [tokenizer.pad_token_id]], axis=0))


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
dataset = dataset.shuffle(1024).batch(32)

In [6]:
optimizer = Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')

In [ ]:
model.fit(dataset, epochs=1)


 48/117 [===========>..................] - ETA: 3:07:36 - loss: 10.8244

In [ ]:
model.save_pretrained('fine_tuned_gpt2')

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained('fine_tuned_gpt2')

In [ ]:
import tensorflow as tf

def generate_text(prompt, max_length=50, temperature=0.7, top_k=50):
    generated_text = prompt
    input_ids = tokenizer.encode(prompt, return_tensors="tf")

    for _ in range(max_length):
        outputs = model(input_ids)
        predictions = outputs.logits[:, -1, :]

        predictions = predictions / temperature

        top_k_values, top_k_indices = tf.nn.top_k(predictions, k=top_k)
        values = tf.nn.softmax(top_k_values, axis=-1)
        sampled_token_index = tf.random.categorical(tf.math.log(values), num_samples=1)
        sampled_token_id = tf.gather(top_k_indices, sampled_token_index, batch_dims=1).numpy().squeeze()

        if sampled_token_id == tokenizer.eos_token_id:
            break

        generated_text += tokenizer.decode([sampled_token_id])
        input_ids = tf.concat([input_ids, [[sampled_token_id]]], axis=-1)

    return generated_text

In [ ]:
prompt = " "
generated_text = generate_text(prompt, max_length=100)
print("Generated text:", generated_text)